In [3]:
import pandapower as pp
import pandapower.networks as nw

# functions we need for creating J
import numpy as np
from pandapower.pf.run_newton_raphson_pf import _get_pf_variables_from_ppci
from pandapower.pd2ppc import _pd2ppc
from pandapower.pf.newtonpf import _create_J_without_numba

# Abbreviations: Jacobian = J, admittance matrix = Ybus


def get_Ybus(net):
    """
    This functions returns the admittance matrix (Ybus) from internal pandapower calculations
    
    """

    # run a power flow to get the ppc (which contains Ybus afterwards)
    pp.runpp(net)

    # ppc (this is similar to the PYPOWER format file, with some changes) contains Ybus
    ppc = net._ppc

    # Get stored Ybus in ppc
    Ybus = ppc["internal"]["Ybus"]
    
    return Ybus


    
def get_J(net, V=None):

    _, ppci = _pd2ppc(net)
    baseMVA, bus, gen, branch, ref, pv, pq, _, _, V0 = _get_pf_variables_from_ppci(ppci)
        
    Ybus = get_Ybus(net)
    if V is None:
        V = V0
    Ibus = np.zeros(len(V))
    pvpq = np.r_[pv, pq]
    
    J = _create_J_without_numba(Ybus, V, pvpq, pq, Ibus=Ibus)
    
    return J
    

if __name__ == "__main__":
    # load example network of which you want the Ybus and J
    net = nw.case9()
    Ybus = get_Ybus(net)
    print("This is Ybus:\n")
    print(Ybus)
    
    J = get_J(net)
    print("This is J with V0;\n")
    print(J)






This is Ybus:

  (0, 0)	-1736.11111111j
  (0, 3)	(-0+1736.11111111j)
  (1, 7)	1600j
  (1, 1)	-1600j
  (2, 2)	-1706.48464164j
  (2, 5)	(-0+1706.48464164j)
  (3, 3)	(330.737896203-3930.88887261j)
  (3, 4)	(-194.219124871+1051.06820519j)
  (3, 0)	1736.11111111j
  (3, 8)	(-136.518771331+1160.40955631j)
  (4, 4)	(322.420038714-1584.09270142j)
  (4, 5)	(-128.200913842+558.824496236j)
  (4, 3)	(-194.219124871+1051.06820519j)
  (5, 5)	(243.709661931-3215.38618051j)
  (5, 6)	(-115.508748089+978.427042636j)
  (5, 4)	(-128.200913842+558.824496236j)
  (5, 2)	1706.48464164j
  (6, 6)	(277.220995414-2330.32490233j)
  (6, 7)	(-161.712247325+1369.79785969j)
  (6, 5)	(-115.508748089+978.427042636j)
  (7, 7)	(280.472685254-3544.56131302j)
  (7, 1)	(-0+1600j)
  (7, 8)	(-118.760437929+597.513453331j)
  (7, 6)	(-161.712247325+1369.79785969j)
  (8, 8)	(255.27920926-1733.82300964j)
  (8, 3)	(-136.518771331+1160.40955631j)
  (8, 7)	(-118.760437929+597.513453331j)
This is J with V0;

  (0, 0)	1600.0
  (0, 6)	-1